In [3]:
import pandas as pd 
import numpy as np
import os 
import json 

c:\Users\91790\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [14]:
text_df, global_metrics_df = pd.read_csv('data/structured/text_data.csv'), pd.read_csv('data/structured/global_metrics_df.csv')

In [15]:
text_df.sample(3)

,text_data
33629,"A guest booked a Resort Hotel with 2 adults, 1..."
71243,"A guest booked a City Hotel with 1 adults, 0.0..."
57933,"A guest booked a City Hotel with 2 adults, 0.0..."


In [16]:
global_metrics_df.sample(3)

,text_data
215,The country ZAF had a total of 31 cancellations.
314,The True season had a total of 39247 bookings.
64,The country BLR had a total of 9 cancellations.


In [17]:
# Merge the two dataframes on the 'text_data' column using concat 
data = pd.concat([global_metrics_df, text_df], axis=0).reset_index(drop=True)

In [18]:
data

,text_data
0,"In 2015, the total revenue for the month of Ju..."
1,"In 2015, the total revenue for the month of Au..."
2,"In 2015, the total revenue for the month of Se..."
3,"In 2015, the total revenue for the month of Oc..."
4,"In 2015, the total revenue for the month of No..."
...,...
119732,"A guest booked a City Hotel with 2 adults, 0.0..."
119733,"A guest booked a City Hotel with 3 adults, 0.0..."
119734,"A guest booked a City Hotel with 2 adults, 0.0..."
119735,"A guest booked a City Hotel with 2 adults, 0.0..."


In [19]:
data.isna().sum()

text_data    53
dtype: int64

In [20]:
# Remove null values from data dataframe 
data = data.dropna()

In [21]:
data.isna().sum()

text_data    0
dtype: int64

In [22]:
text_data = data['text_data'].tolist()

In [23]:
text_data[:5]

['In 2015, the total revenue for the month of July was 588400.00 USD. This reflects the booking revenue generated by the hotel during this period.',
 'In 2015, the total revenue for the month of August was 898091.00 USD. This reflects the booking revenue generated by the hotel during this period.',
 'In 2015, the total revenue for the month of September was 917673.00 USD. This reflects the booking revenue generated by the hotel during this period.',
 'In 2015, the total revenue for the month of October was 716712.00 USD. This reflects the booking revenue generated by the hotel during this period.',
 'In 2015, the total revenue for the month of November was 236721.00 USD. This reflects the booking revenue generated by the hotel during this period.']

---
Creating Embeddings

In [9]:
from sentence_transformers import SentenceTransformer

In [33]:
import torch

print(torch.cuda.is_available())  # Should return True if GPU is detected
print(torch.cuda.device_count())  # Number of GPUs detected
# print(torch.cuda.get_device_name(0))  # Name of the first GPU

False
0


In [11]:
embedder = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

In [36]:
embeddings = embedder.encode(text_data, batch_size=16, show_progress_bar=True)

Batches:   0%|          | 0/7481 [00:00<?, ?it/s]

In [41]:
# Save these embeddings to disk
np.save('data/structured/embeddings.npy', embeddings)

---

### Load Saved Embeddings

In [4]:
# load the embeddings from disk
embeddings = np.load('data/structured/embeddings.npy')

In [118]:
print(type(embedder))

<class 'sentence_transformers.SentenceTransformer.SentenceTransformer'>


In [5]:
embeddings

array([[ 0.0766786 ,  0.01734373, -0.02381854, ..., -0.11824384,
        -0.03148815, -0.09610205],
       [ 0.0490954 ,  0.01373599, -0.03305401, ..., -0.1166638 ,
        -0.05858671, -0.10482675],
       [ 0.0535932 , -0.02918408, -0.03687486, ..., -0.13470587,
        -0.05519529, -0.07855507],
       ...,
       [ 0.03991789,  0.05281889, -0.04423684, ...,  0.01935323,
        -0.09267406, -0.04211508],
       [ 0.04451197,  0.06263656, -0.04673076, ...,  0.0162664 ,
        -0.09152321, -0.04120836],
       [ 0.04052964,  0.05498633, -0.05035856, ...,  0.02399173,
        -0.08753999, -0.04146033]], dtype=float32)

---

### Uploading Embeddings to index 

In [144]:
PINECONE_API_KEY = ""

In [6]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=PINECONE_API_KEY)

Creating Index

In [9]:
index_name = "hotelbookings"

pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

In [10]:
pc.list_indexes()

{'indexes': [{'deletion_protection': 'disabled',
              'dimension': 384,
              'host': 'hotelbookings-tqz6la2.svc.aped-4627-b74a.pinecone.io',
              'metric': 'cosine',
              'name': 'hotelbookings',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
              'status': {'ready': True, 'state': 'Ready'}}]}

In [25]:
for e in embeddings:
    print(type(e))
    break

<class 'numpy.ndarray'>


In [27]:
vectors = []

# PineCone Format 
num = 1
for t, e in zip(text_data, embeddings):
    vectors.append(
        {
            "id": f"vector_{num}",
            "values": e, 
            "metadata": {"text": t}
        }
    )
    num += 1

In [32]:
# Wait for the index to be ready

while not pc.describe_index(index_name).status['ready']:

    time.sleep(1)

In [33]:
index = pc.Index(index_name)

Upserting

In [38]:
from tqdm import tqdm

# Define a smaller batch size
BATCH_SIZE = 50  

# Progress bar setup
total_batches = len(vectors) // BATCH_SIZE + (len(vectors) % BATCH_SIZE > 0)  # Total batch count
print(f"Total batches: {total_batches}")

Total batches: 2394


In [39]:
print(f"Uploading {len(vectors)} vectors in {total_batches} batches...")

# Upload in smaller batches with progress tracking
for i in tqdm(range(0, len(vectors), BATCH_SIZE), desc="Uploading vectors", unit="batch"):
    batch = vectors[i : i + BATCH_SIZE]
    index.upsert(vectors=batch)

print("✅ Upload complete!")

Uploading 119684 vectors in 2394 batches...


Uploading vectors: 100%|██████████| 2394/2394 [24:20<00:00,  1.64batch/s]

✅ Upload complete!


In [40]:
print(index.describe_index_stats())

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 119684}},
 'total_vector_count': 119684}


In [112]:
query = "Total revenue of May in 2017?"

In [52]:
# Convert this query to an embedding
from sentence_transformers import SentenceTransformer

In [113]:
embedder = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
query_embedding = embedder.encode([query])[0]

In [114]:
query_embedding.shape 

(384,)

In [115]:
query_embedding[:5]

array([ 0.0506124 ,  0.00376086,  0.01435805, -0.05173252, -0.00770749],
      dtype=float32)

---

In [92]:
from pinecone.grpc import PineconeGRPC as Pinecone

In [93]:
pc = Pinecone(api_key=PINECONE_API_KEY)

In [94]:
index = pc.Index(host="https://hotelbookings-tqz6la2.svc.aped-4627-b74a.pinecone.io")

In [95]:
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 119684}},
 'total_vector_count': 119684}

In [116]:
response = index.query(
    vector=query_embedding,
    top_k=2,
    # include_values=True,
    include_metadata=True
)
    
print(response)

{'matches': [{'id': 'vector_23',
              'metadata': {'text': 'In 2017, the total revenue for the month '
                                   'of May was 1488452.00 USD. This reflects '
                                   'the booking revenue generated by the hotel '
                                   'during this period.'},
              'score': 0.7673542,
              'sparse_values': {'indices': [], 'values': []},
              'values': []},
             {'id': 'vector_11',
              'metadata': {'text': 'In 2016, the total revenue for the month '
                                   'of May was 1043009.00 USD. This reflects '
                                   'the booking revenue generated by the hotel '
                                   'during this period.'},
              'score': 0.72360766,
              'sparse_values': {'indices': [], 'values': []},
              'values': []}],
 'namespace': '',
 'usage': {'read_units': 6}}


In [125]:
matches = response.matches

In [130]:
context = []
for i in matches:
    context.append(i.metadata['text'])

In [131]:
context

['In 2017, the total revenue for the month of May was 1488452.00 USD. This reflects the booking revenue generated by the hotel during this period.',
 'In 2016, the total revenue for the month of May was 1043009.00 USD. This reflects the booking revenue generated by the hotel during this period.']

---
LLM

In [143]:
groq_api = ""

In [139]:
import groq

In [140]:
import os

from groq import Groq

client = Groq(
    api_key=groq_api,
)

In [142]:
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Explain the importance of fast language models",
        }
    ],
    model="llama3-70b-8192",
)

print(chat_completion.choices[0].message.content)

Fast language models have gained significant importance in recent years due to their ability to process and generate human-like language at incredible speeds. Here are some reasons why fast language models are crucial:

1. **Real-time Applications**: Fast language models enable real-time applications such as chatbots, virtual assistants, and language translation systems to respond quickly and efficiently. This improves the user experience, making interactions more natural and seamless.
2. **Improved Productivity**: Fast language models can automate tasks such as content generation, summarization, and language translation, freeing up human resources for more creative and high-value tasks. This leads to increased productivity and efficiency.
3. **Enhanced Customer Experience**: Fast language models can help businesses respond quickly to customer inquiries, providing timely support and improving customer satisfaction. This can lead to increased customer loyalty and retention.
4. **Competi